In [1]:
few_shot_path = '/scratch/dzhang5/LLM/TWEET-FID/expert.train.short.csv'
data_path = '/scratch/dzhang5/LLM/TWEET-FID/expert.smalltest.csv'
output_dir = '/scratch/dzhang5/LLM/TWEET-FID/test-results-autolabel-all-ae'
model_name = "gpt-3.5-turbo"
text_column = 'context'
explanation_column = 'entity_explanation'
example_selection_label_column = 'has_all'
label_symbol = "^^^^"
few_shot_num = 8
few_shot_selection = 'label_diversity_similarity'
use_current_explanation = False
use_ground_explanation = False
token_path = "/home/dzhang5/.cache/huggingface/token"
cache=False
console_output=True
temperature=0.1
label_version='v1'
task_version='v1'
random_shuffle_examples = True
random_shuffle_examples_seed = 1

# Generation Stage

In [2]:
from autolabel.schema import ModelProvider, TaskType
from autolabel.models import register_model, MODEL_REGISTRY
from hf_pipeline_new import HFPipelineLLMNew
from few_shot_new import NewAutoLabelConfig, NewExampleSelectorFactory
from autolabel.few_shot import ExampleSelectorFactory
from template_inst import update_inst_mode
from named_entity_recognition_new import NewNamedEntityRecognitionTask
from classification_new import NewClassificationTask
from question_answering_new import NewQuestionAnsweringTask
from attribute_extraction_new import NewAttributeExtractionTask
from autolabel.tasks import TASK_TYPE_TO_IMPLEMENTATION 
from prompt_template import load_all_in_one_prompt
from dataset_new import process_labels
import sys

In [3]:
update_inst_mode(model_name)
TASK_TYPE_TO_IMPLEMENTATION[TaskType.NAMED_ENTITY_RECOGNITION] = NewNamedEntityRecognitionTask
TASK_TYPE_TO_IMPLEMENTATION[TaskType.CLASSIFICATION] = NewClassificationTask
TASK_TYPE_TO_IMPLEMENTATION[TaskType.QUESTION_ANSWERING] = NewQuestionAnsweringTask
TASK_TYPE_TO_IMPLEMENTATION[TaskType.ATTRIBUTE_EXTRACTION] = NewAttributeExtractionTask
sys.modules['autolabel.labeler'].ExampleSelectorFactory = NewExampleSelectorFactory
sys.modules['autolabel.dataset'].AutolabelDataset.process_labels = process_labels
register_model(ModelProvider.HUGGINGFACE_PIPELINE, HFPipelineLLMNew)

In [4]:
from autolabel import LabelingAgent, AutolabelDataset
import json
import os
import pandas as pd

In [5]:
with open(token_path) as tfile:
    token_str = tfile.read()

from huggingface_hub import login
login(token=token_str)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/dzhang5/.cache/huggingface/token
Login successful


In [6]:
no_auto = ["microsoft/prophetnet", "microsoft/phi-2", "google/pegasus-x"]
if any([pre in model_name for pre in no_auto]):
    device_map = None
else:
    device_map = "auto"

In [7]:
if not os.path.exists(output_dir):
    # Create the directory
    os.makedirs(output_dir)
output_name = os.path.split(model_name)[-1] + '_' + few_shot_selection + '_COT_AE_' + str(explanation_column) + '_cur_' + str(use_current_explanation) + '_ground_' + str(use_ground_explanation) + '_' + os.path.split(data_path)[-1]
output_path = os.path.join(output_dir, output_name)

In [8]:
refuel_models = ["refuel-llm", "llama-13b-chat"]
if model_name in refuel_models:
    provider = "refuel"
    em_provider = "huggingface_pipeline"
    model_params = {"max_length":4096, "temperature": temperature}
    task_name = f"FoodborneIllnessIncidentTweetNERAE_{few_shot_selection}_{model_name}"
elif model_name.startswith('gpt'):
    provider = "openai"
    em_provider = "openai"
    model_params = {"max_tokens":4096, "temperature": temperature}
    task_name = f"FoodborneIllnessIncidentTweetNERAE_{few_shot_selection}_{model_name}"
else:
    provider = "huggingface_pipeline"
    em_provider = "huggingface_pipeline"
    model_params = {"max_length":4096, "temperature": temperature,
                    "quantize": 16, "device_map": device_map,
                    "token": token_str}
    task_name = f"FoodborneIllnessIncidentTweetNERAE_{few_shot_selection}_{model_name.split('/')[1]}"

In [10]:
sym_len = len(label_symbol)
label_prefix, label_suffix, label_description, task_guideline, output_guideline = load_all_in_one_prompt(label_symbol, label_version, task_version, True)

In [11]:
test_data = pd.read_csv(data_path)

In [12]:
if use_current_explanation and not use_ground_explanation:
    explanation_column = explanation_column + "_prediction"

In [13]:
attribute_description = [{'name': k+'_answer', 'description': f'{k} entities {v}'} for k, v in label_description.items()]

In [15]:
attribute_description.append({'name':'sentence_class', 'description': 'If the text describes a potential foodborne illness event','options':['Yes', 'No']})

In [16]:
config = {
    "task_name": task_name,
    "task_type": "attribute_extraction",
    "dataset": {
        "text_column": text_column,
        "explanation_column": explanation_column,
        "example_selection_label_column": example_selection_label_column,
        "delimiter": ",",
        "label_description": label_description 
    },
    "model": {
        "provider": provider,
        "name": model_name,
        "params": model_params
    },
    "embedding": {
        "provider": em_provider,
    },
    "prompt": {
        "task_guidelines": task_guideline,
        "output_guidelines": output_guideline,
        "attributes": attribute_description,
        "example_selection_labels":[
            "yes",
            "no"
        ],
        "few_shot_examples": few_shot_path,
        "few_shot_selection": few_shot_selection,
        "few_shot_num": few_shot_num,
        "use_current_explanation": use_current_explanation,
        "random_shuffle_examples": random_shuffle_examples,
        "random_shuffle_examples_seed": random_shuffle_examples_seed,
        "example_template": f"Input: {{{text_column}}}\nOutput: Let's think step by step.\n{{{explanation_column}}}\n"+"{output_dict}",
        "chain_of_thought": True
    }
}

config = NewAutoLabelConfig(config)

In [17]:
agent = LabelingAgent(config=config, console_output=console_output, cache=cache)

2024-04-04 22:59:15 autolabel.labeler WARNING: cache parameter is deprecated and will be removed soon. Please use generation_cache and transform_cache instead.
/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [18]:
label_columns = [i['name'] for i in attribute_description]

In [19]:
if use_current_explanation:
    ds = AutolabelDataset(test_data[[text_column, explanation_column, example_selection_label_column] + label_columns], config=config)
else:
    ds = AutolabelDataset(test_data[[text_column, example_selection_label_column] + label_columns], config=config)

In [20]:
agent.plan(ds)

/scratch/dzhang5/visa/visa39/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
2024-04-04 22:59:53 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-04-04 23:00:00 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-04-04 23:00:05 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Output()

2024-04-04 23:00:08 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-04-04 23:00:09 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-04-04 23:00:09 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-04-04 23:00:09 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-04-04 23:00:10 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-04-04 23:00:10 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-04-04 23:00:10 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-04-04 23:00:11 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-04-04 23:00:11 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-04-04 23:00:11 httpx INFO: HTTP Request: 

┌──────────────────────────┬─────────┐
│ Total Estimated Cost     │ $0.1124 │
│ Number of Examples       │ 10      │
│ Average cost per example │ $0.0112 │
└──────────────────────────┴─────────┘

───────────────────────────────────────────────── Prompt Example ──────────────────────────────────────────────────

You are an expert at extracting entities that are related to foodborne illness incident from text. In the given 
text, your task is to label multiple types of entity that are related to foodborne illness incident, and answer if 
the text describes a potential foodborne illness incident event. Note that you should surround the extracted 
entities in the text with "^^" and "^^". Use the following examples as a guide for your predictions and format your
responses similarly.

Your answer will consist of an explanation, followed by the correct labeled sentences for all types of entity and 
the answer to the text relevance question in json format. Note that if the given text does not include any certain 
type entity related to foodborne illness, your labeled sentence for this type of entity will be the input text.The 
keys of the json output and the definitions for all type of relevant entity are provided below: 
{
    "Food_answer": "Food entities are specific food item that caused the potential foodborne illness incident.",
    "Symptom_answer": "Symptom entities are specific symptoms experienced by the affected person as a result of the
suspected foodborne illness.",
    "Location_answer": "Location entities pretrain to the location where the affected person purchased or acquired 
the food associated with the potential foodborne illness.",
    "Keyword_answer": "Keyword entities include other relevant keywords or terms associated with foodborne 
illnesses, such as \"food poisoning\".",
    "sentence_class": "If the text describes a potential foodborne illness event\nOptions:\nYes,No"
}

Some examples with their output answers are provided below:

Input: @USER @USER @USER Flu , food poisoning , hangover I dont care , gimme MJ
Output: Let's think step by step.
The text does not indicate a foodborne illness incident. So, no entity in the text is related to foodborne illness 
incident.
{"Food_answer": "@USER @USER @USER Flu , food poisoning , hangover I dont care , gimme MJ", "Symptom_answer": 
"@USER @USER @USER Flu , food poisoning , hangover I dont care , gimme MJ", "Location_answer": "@USER @USER @USER 
Flu , food poisoning , hangover I dont care , gimme MJ", "Keyword_answer": "@USER @USER @USER Flu , food poisoning 
, hangover I dont care , gimme MJ", "sentence_class": "No"}

Input: @USER @USER well I've been off with food poisoning , so if you fancy sick in your face , come get it ! ! ; )
Output: Let's think step by step.
The text indicates a foodborne illness incident. No Food entity in the text is relevant to foodborne illness 
incident.No Location entity in the text is relevant to foodborne illness incident.No Symptom entity in the text is 
relevant to foodborne illness incident.The word "food poisoning" in the text is a Keyword entity, which is other 
relevant keyword or term associated with foodborne illnesses, such as "food poisoning". 
{"Food_answer": "@USER @USER well I've been off with food poisoning , so if you fancy sick in your face , come get 
it ! ! ; )", "Symptom_answer": "@USER @USER well I've been off with food poisoning , so if you fancy sick in your 
face , come get it ! ! ; )", "Location_answer": "@USER @USER well I've been off with food poisoning , so if you 
fancy sick in your face , come get it ! ! ; )", "Keyword_answer": "@USER @USER well I've been off with ^^food 
poisoning^^ , so if you fancy sick in your face , come get it ! ! ; )", "sentence_class": "Yes"}

Input: Think I might have gotten a mild case of food-poisoning today - so that's been super fun .
Output: Let's think step by step.
The text does not indicate a foodborne illness incident. So, no entity in the text is related to foodborne illness 
incident.
{"Food_answer": "Think I might have gotten a mild case of food-poisoning today - so that's been super fun .", 
"Symptom_answer": "Think I might have gotten a mild case of food-poisoning today - so that's been super fun .", 
"Location_answer": "Think I might have gotten a mild case of food-poiso

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [21]:
# now, do the actual labeling
ds = agent.run(ds)

Output()

2024-04-04 23:01:04 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-04-04 23:01:09 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-04 23:01:09 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-04-04 23:01:12 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-04 23:01:13 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-04-04 23:01:18 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-04 23:01:18 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-04-04 23:01:20 httpx INFO: HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-04-04 23:01:20 httpx INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-04-04 23:01:24 ht

Actual Cost: 0.0354

┏━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━┓
┃ Food… ┃ Food… ┃ Food… ┃ Symp… ┃ Sym… ┃ Symp… ┃ Loc… ┃ Loca… ┃ Loc… ┃ Keyw… ┃ Key… ┃ Keyw… ┃ sen… ┃ sent… ┃ sen… ┃
┡━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━╇━━━━━━━╇━━━━━━╇━━━━━━━╇━━━━━━╇━━━━━━━╇━━━━━━╇━━━━━━━╇━━━━━━╇━━━━━━━╇━━━━━━┩
│ 10    │ 1.0   │ 0.9   │ 10    │ 1.0  │ 0.9   │ 10   │ 1.0   │ 0.8  │ 10    │ 1.0  │ 0.9   │ 10   │ 1.0   │ 0.8  │
└───────┴───────┴───────┴───────┴──────┴───────┴──────┴───────┴──────┴───────┴──────┴───────┴──────┴───────┴──────┘

In [22]:
metrics = ds.eval()

┏━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━┓
┃ Food… ┃ Food… ┃ Food… ┃ Symp… ┃ Sym… ┃ Symp… ┃ Loc… ┃ Loca… ┃ Loc… ┃ Keyw… ┃ Key… ┃ Keyw… ┃ sen… ┃ sent… ┃ sen… ┃
┡━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━━╇━━━━━━╇━━━━━━━╇━━━━━━╇━━━━━━━╇━━━━━━╇━━━━━━━╇━━━━━━╇━━━━━━━╇━━━━━━╇━━━━━━━╇━━━━━━┩
│ 10    │ 1.0   │ 0.9   │ 10    │ 1.0  │ 0.9   │ 10   │ 1.0   │ 0.8  │ 10    │ 1.0  │ 0.9   │ 10   │ 1.0   │ 0.8  │
└───────┴───────┴───────┴───────┴──────┴───────┴──────┴───────┴──────┴───────┴──────┴───────┴──────┴───────┴──────┘

In [23]:
ds.df.to_csv(output_path, index=False)
ds.df.to_pickle(output_path.replace('.csv', '.pkl'))